In [1]:
#####################
# AUTO LOGIN-1 of 1 #
#####################

import login
from login import *
import pandas as pd

KFw2z8IumHn8N7s81sslYUKumasRL196


In [2]:
#####################
#    PARAMENTERS    #
#####################

this_month_token = 690691
next_month_token = 780803

this_month_symbol = 'USDINR20OCTFUT'
next_month_symbol = 'USDINR20NOVFUT'

buysell_diff = 0.2100                                 # 0.2100 per day
squareup_diff = 0.1950                                # 0.1950 per day

fullquant = 400

order_type = 'MIS'

In [3]:
symbol_ip = this_month_symbol
symbol_ip2 = next_month_symbol

inst_token = this_month_token
inst_token2 = next_month_token

myquantity = 0
myquantity2 = 0

def getquant():
    global myquantity,myquantity2
    global order_type
    global symbol_ip
    allpos = kite.positions()['net']
    for i in range(len(allpos)):
        if(allpos[i]['tradingsymbol']==symbol_ip and  allpos[i]['product']== order_type):
            myquantity = allpos[i]['quantity']
            print('My Quantity 1 : ' + str(allpos[i]['quantity']))
        if(allpos[i]['tradingsymbol']==symbol_ip2 and  allpos[i]['product']== order_type):
            myquantity2 = allpos[i]['quantity']
            print('My Quantity 2 : ' + str(allpos[i]['quantity']))

getquant()

def placeneworder(quantdiff,symbol_ipp):
    global stopbuy
    global stopsell
    global order_type
#     global symbol_ip,symbol_ip2
    if(quantdiff>0):
        try:
                order_id= kite.place_order(tradingsymbol=symbol_ipp,
                            exchange=kite.EXCHANGE_CDS,
                            transaction_type=kite.TRANSACTION_TYPE_BUY,
                            quantity=abs(quantdiff),
                            order_type=kite.ORDER_TYPE_MARKET,
                            variety = kite.VARIETY_REGULAR,
                            product=kite.PRODUCT_MIS if order_type=='MIS' else kite.PRODUCT_NRML)
                getquant()
        except Exception as e:
            print(e)
            getquant()
    if(quantdiff<0):
        try:
                order_id= kite.place_order(tradingsymbol=symbol_ipp,
                            exchange=kite.EXCHANGE_CDS,
                            transaction_type=kite.TRANSACTION_TYPE_SELL,
                            quantity=abs(quantdiff),
                            order_type=kite.ORDER_TYPE_MARKET,
                            variety = kite.VARIETY_REGULAR,
                            product=kite.PRODUCT_MIS if order_type=='MIS' else kite.PRODUCT_NRML)
                getquant()
        except Exception as e:
            print(e)
            getquant()


def ckqnt (orderquant):
    global myquantity,myquantity2,symbol_ip,symbol_ip2
    quantdiff = orderquant - myquantity
    quantdiff2 = -orderquant - myquantity2
    
    placeneworder(quantdiff,symbol_ip)
    placeneworder(quantdiff2,symbol_ip2)
    return myquantity

In [ ]:
#################
# START TRADING #
#################

import logging
from kiteconnect import KiteTicker
logging.basicConfig(level=logging.DEBUG)
# kws = KiteTicker("w19o0chuo929jxkp", "eA5B5OJQNOtZ0OYihkBmYw7Ke3B9pmCC")

def on_ticks(ws, ticks):
    #print(type(ticks[0]['last_price']))
    print('==========================')
    print( 'LTP1 : ' + str(ticks[0]['last_price']))
    print( 'LTP2 : ' + str(ticks[1]['last_price']))
    ltp_diffs = ticks[1]['last_price'] - ticks[0]['last_price']
    print( 'Difference : ' + str(ltp_diffs))
    if(ltp_diffs==buysell_diff):
        ckqnt(int(fullquant/2))
    if(ltp_diffs==squareup_diff):
        ckqnt(0)
#     print('==========================')
#     print()

def on_connect(ws, response):
    global inst_token,inst_token2
    ws.subscribe([inst_token,inst_token2])
    ws.set_mode(ws.MODE_FULL, [inst_token,inst_token2])

def on_close(ws, code, reason):
    ws.stop()
    
def on_error(ws, code, reason):
    logging.error("closed connection on error: {} {}".format(code, reason))

def on_noreconnect(ws):
    logging.error("Reconnecting the websocket failed")

def on_reconnect(ws, attempt_count):
    logging.debug("Reconnecting the websocket: {}".format(attempt_count))

#def on_order_update(ws, data):
#    print("order update: ", data)

kws.on_error = on_error
kws.on_noreconnect = on_noreconnect
kws.on_reconnect = on_reconnect
#kws.on_order_update = on_order_update

kws.on_ticks = on_ticks
kws.on_connect = on_connect
kws.on_close = on_close

kws.connect()